**Problem 5**
We are given the following joint distribution over the random variables A, B, C, and D. 

| A | B | C | D | P(A,B,C,D) |
|---|---|---|---|------------|
| T | T | T | T | 0.0448     |
| T | T | T | F | 0.0252     |
| T | T | F | T | 0.0112     |
| T | T | F | F | 0.0588     |
| T | F | T | T | 0.0144     |
| T | F | T | F | 0.0144     |
| T | F | F | T | 0.096      |
| F | F | F | F | 0.216      |
| F | T | T | T | 0.1024     |
| F | T | T | F | 0.0576     |
| F | T | F | T | 0.0256     |
| F | T | F | F | 0.1344     |
| F | F | T | T | 0.1152     |
| F | F | T | F | 0.1152     |
| F | F | F | T | 0.0768     |
| F | F | F | F | 0.1728     |

 1. Are A and B independent? 
 2. Are A and C independent?
 3. Are A and C independent given B?
 4. Are A and D independent?
 5. Are A and D independent given C?


In [15]:
import itertools
from functools import reduce
from operator import mul
import matplotlib.pyplot as plt
import networkx as nx
from daft import PGM
from pgmpy.models import BayesianModel, BayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator, ParameterEstimator
from pgmpy.factors.discrete import TabularCPD, JointProbabilityDistribution, DiscreteFactor
from pgmpy.inference import VariableElimination
import pandas as pd
import numpy as np
from pgmpy.independencies import Independencies


# Define the variables and their states
variables = ['A', 'B', 'C', 'D']
cardinality = [2, 2, 2, 2]  # Two states for each variable: True and False

# Define the probabilities corresponding to the combinations in your table
probabilities = [
    0.0448, 0.0252, 0.0112, 0.0588,
    0.0144, 0.0144, 0.0096, 0.0216,
    0.1024, 0.0576, 0.0256, 0.1344,
    0.1152, 0.1152, 0.0768, 0.1728
]

# Create the Joint Probability Distribution
jpd_table = JointProbabilityDistribution(variables, cardinality, np.array(probabilities))
print(jpd_table)

+------+------+------+------+--------------+
| A    | B    | C    | D    |   P(A,B,C,D) |
+======+======+======+======+==============+
| A(0) | B(0) | C(0) | D(0) |       0.0448 |
+------+------+------+------+--------------+
| A(0) | B(0) | C(0) | D(1) |       0.0252 |
+------+------+------+------+--------------+
| A(0) | B(0) | C(1) | D(0) |       0.0112 |
+------+------+------+------+--------------+
| A(0) | B(0) | C(1) | D(1) |       0.0588 |
+------+------+------+------+--------------+
| A(0) | B(1) | C(0) | D(0) |       0.0144 |
+------+------+------+------+--------------+
| A(0) | B(1) | C(0) | D(1) |       0.0144 |
+------+------+------+------+--------------+
| A(0) | B(1) | C(1) | D(0) |       0.0096 |
+------+------+------+------+--------------+
| A(0) | B(1) | C(1) | D(1) |       0.0216 |
+------+------+------+------+--------------+
| A(1) | B(0) | C(0) | D(0) |       0.1024 |
+------+------+------+------+--------------+
| A(1) | B(0) | C(0) | D(1) |       0.0576 |
+------+--

In [21]:
import pandas as pd

# Define your JPD data
jpd_data = [
    ('A(0)', 'B(0)', 'C(0)', 'D(0)', 0.0448),
    ('A(0)', 'B(0)', 'C(0)', 'D(1)', 0.0252),
    ('A(0)', 'B(0)', 'C(1)', 'D(0)', 0.0112),
    ('A(0)', 'B(0)', 'C(1)', 'D(1)', 0.0588),
    ('A(0)', 'B(1)', 'C(0)', 'D(0)', 0.0144),
    ('A(0)', 'B(1)', 'C(0)', 'D(1)', 0.0144),
    ('A(0)', 'B(1)', 'C(1)', 'D(0)', 0.0096),
    ('A(0)', 'B(1)', 'C(1)', 'D(1)', 0.0216),
    ('A(1)', 'B(0)', 'C(0)', 'D(0)', 0.1024),
    ('A(1)', 'B(0)', 'C(0)', 'D(1)', 0.0576),
    ('A(1)', 'B(0)', 'C(1)', 'D(0)', 0.0256),
    ('A(1)', 'B(0)', 'C(1)', 'D(1)', 0.1344),
    ('A(1)', 'B(1)', 'C(0)', 'D(0)', 0.1152),
    ('A(1)', 'B(1)', 'C(0)', 'D(1)', 0.1152),
    ('A(1)', 'B(1)', 'C(1)', 'D(0)', 0.0768),
    ('A(1)', 'B(1)', 'C(1)', 'D(1)', 0.1728)
]

# Create a DataFrame from the JPD data
columns = ['A', 'B', 'C', 'D', 'P(A,B,C,D)']
df = pd.DataFrame(jpd_data, columns=columns)

# Initialize dictionaries to hold sums for A(0) and A(1)
sums_A0 = {}
sums_A1 = {}

# Iterate over rows in the DataFrame and sum up probabilities for A(0) and A(1)
for index, row in df.iterrows():
    a_val = row['A']  # Get the value of A from the row
    probability = row['P(A,B,C,D)']  # Get the probability from the row

    # Check if A(0) or A(1)
    if a_val == 'A(0)':
        sums_A0[a_val] = sums_A0.get(a_val, 0) + probability
    else:
        sums_A1[a_val] = sums_A1.get(a_val, 0) + probability

# Print the sums
print("Sum of probabilities for A(0):")
for key, value in sums_A0.items():
    print(f"{key}: {value}")

print("\nSum of probabilities for A(1):")
for key, value in sums_A1.items():
    print(f"{key}: {value}")


Sum of probabilities for A(0):
A(0): 0.2

Sum of probabilities for A(1):
A(1): 0.8
